#Iain Doucette
Protein design using Colab Design notebook linked below. Minimal changes were made. PDB: https://www.rcsb.org/structure/6aru file was changed to include just the EGFR sequence and not the Cetuximab sequence

ColabDesign/af/examples/af_relax_design.ipynb


In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install pyppeteer nest_asyncio")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model

from IPython.display import HTML
from google.colab import files
import numpy as np

import requests, time
if not os.path.isfile("TMscore"):
  os.system("wget -qnc https://zhanggroup.org/TM-score/TMscore.cpp")
  os.system("g++ -static -O3 -ffast-math -lm -o TMscore TMscore.cpp")
def tmscore(x,y):
  # pass to TMscore
  output = os.popen(f'./TMscore {x} {y}')
  # parse outputs
  parse_float = lambda x: float(x.split("=")[1].split()[0])
  o = {}
  for line in output:
    line = line.rstrip()
    if line.startswith("RMSD"): o["rms"] = parse_float(line)
    if line.startswith("TM-score"): o["tms"] = parse_float(line)
    if line.startswith("GDT-TS-score"): o["gdt"] = parse_float(line)
  return o

import asyncio
import nest_asyncio
from pyppeteer import launch
import base64

# Apply nest_asyncio to enable nested event loops
nest_asyncio.apply()

async def fetch_blob_content(page, blob_url):
  blob_to_base64 = """
  async (blobUrl) => {
      const blob = await fetch(blobUrl).then(r => r.blob());
      return new Promise((resolve) => {
          const reader = new FileReader();
          reader.onloadend = () => resolve(reader.result);
          reader.readAsDataURL(blob);
      });
  }
  """
  base64_data = await page.evaluate(blob_to_base64, blob_url)
  _, encoded = base64_data.split(',', 1)
  return base64.b64decode(encoded)

async def extract_pdb_file_download_link_and_content(url):
  browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
  page = await browser.newPage()
  await page.goto(url, {'waitUntil': 'networkidle0'})
  elements = await page.querySelectorAll('a.btn.bg-purple')
  for element in elements:
      href = await page.evaluate('(element) => element.getAttribute("href")', element)
      if 'blob:https://esmatlas.com/' in href:
          content = await fetch_blob_content(page, href)
          await browser.close()
          return href, content
  await browser.close()
  return "No PDB file link found.", None
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!      PDB     !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def esmfold_api(sequence):
  url = !wget file:///C:/Users/iaind/Downloads/rcsb_pdb_6ARU_2.fasta
  result = asyncio.get_event_loop().run_until_complete(extract_pdb_file_download_link_and_content(url))
  if result[1]:
      pdb_str = result[1].decode('utf-8')
      return pdb_str
  else:
      return "Failed to retrieve PDB content."

import jax
import jax.numpy as jnp
from colabdesign.af.alphafold.common import residue_constants


CPU times: user 1.94 s, sys: 260 ms, total: 2.2 s
Wall time: 2min 19s


In [ ]:
#@title # hallucination
#@markdown For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).
LENGTH = 250 #@param {type:"integer"}
COPIES = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw"}
MODE = "manuscript" #@param ["original", "manuscript"]
use_rg_loss = True #@param {type:"boolean"}

#@markdown ProteinMPNN Settings
use_mpnn_loss = False #@param {type:"boolean"}
use_solubleMPNN = False #@param {type:"boolean"}
#@markdown

def add_rg_loss(self, weight=0.1):
  '''add radius of gyration loss'''
  def loss_fn(inputs, outputs):
    xyz = outputs["structure_module"]
    ca = xyz["final_atom_positions"][:,residue_constants.atom_order["CA"]]
    if self.protocol == "binder":
      ca = ca[-self._binder_len:]
    if MODE == "manuscript":
      ca = ca[::5]
    rg = jnp.sqrt(jnp.square(ca - ca.mean(0)).sum(-1).mean() + 1e-8)
    if MODE == "original":
      rg_th = 2.38 * ca.shape[0] ** 0.365
      rg = jax.nn.elu(rg - rg_th)
    return {"rg":rg}
  self._callbacks["model"]["loss"].append(loss_fn)
  self.opt["weights"]["rg"] = weight

def add_mpnn_loss(self, mpnn=0.1, mpnn_seq=0.0):
  '''
  add mpnn loss
  mpnn = maximize confidence of proteinmpnn
  mpnn_seq = push designed sequence to match proteinmpnn logits
  '''

  self._mpnn = mk_mpnn_model(weights = "soluble" if use_solubleMPNN else "original")
  def loss_fn(inputs, outputs, aux, key):

    # get structure
    atom_idx = tuple(residue_constants.atom_order[k] for k in ["N","CA","C","O"])
    I = {"S":           inputs["aatype"],
         "residue_idx": inputs["residue_index"],
         "chain_idx":   inputs["asym_id"],
         "X":           outputs["structure_module"]["final_atom_positions"][:,atom_idx],
         "mask":        outputs["structure_module"]["final_atom_mask"][:,1],
         "lengths":     self._lengths,
         "key":         key}

    if "offset" in inputs:
      I["offset"] = inputs["offset"]

    # set autoregressive mask
    L = sum(self._lengths)
    if self.protocol == "binder":
      I["ar_mask"] = 1 - np.eye(L)
      I["ar_mask"][-self._len:,-self._len:] = 0
    else:
      I["ar_mask"] = np.zeros((L,L))

    # get logits
    logits = self._mpnn._score(**I)["logits"][:,:20]
    if self.protocol == "binder":
      logits = logits[-self._len:]
    else:
      logits = logits[:self._len]
    aux["mpnn_logits"] = logits

    # compute loss
    log_q = jax.nn.log_softmax(logits)
    p = inputs["seq"]["hard"]
    q = jax.nn.softmax(logits)
    losses = {}
    losses["mpnn"] = -log_q.max(-1).mean()
    losses["mpnn_seq"] = -(p * jax.lax.stop_gradient(log_q)).sum(-1).mean()
    return losses

  self._callbacks["model"]["loss"].append(loss_fn)
  self.opt["weights"]["mpnn"] = mpnn
  self.opt["weights"]["mpnn_seq"] = mpnn_seq

clear_mem()
af_model = mk_afdesign_model(protocol="hallucination")
af_model.prep_inputs(length=LENGTH, copies=COPIES)

# add extra losses
if use_rg_loss:   add_rg_loss(af_model)
if use_mpnn_loss: add_mpnn_loss(af_model)

print("length",af_model._lengths)
print("weights",af_model.opt["weights"])


length [250]
weights {'con': 1.0, 'exp_res': 0.0, 'helix': 0.0, 'pae': 0.0, 'plddt': 0.0, 'seq_ent': 0.0, 'rg': 0.1}


In [ ]:
af_model.restart()
if MODE == "original":
  # pre-design with gumbel initialization and softmax activation
  af_model.set_weights(plddt=0.0, pae=0.0)
  af_model.set_seq(mode=["gumbel"])
  af_model.design_soft(50)
  af_model.set_seq(af_model.aux["seq"]["pseudo"])

if MODE == "manuscript":
  af_model.set_seq(mode=["gumbel","soft"])

af_model.set_weights(plddt=1.0, pae=1.0)
af_model.design_logits(40)
af_model.design_logits(10, save_best=True)


1 models [1] recycles 0 hard 0 soft 0 temp 1 loss 10.83 pae 0.79 con 4.71 plddt 0.50 ptm 0.19 rg 48.34
2 models [3] recycles 0 hard 0 soft 0 temp 1 loss 8.48 pae 0.74 con 4.07 plddt 0.62 ptm 0.25 rg 32.94
3 models [3] recycles 0 hard 0 soft 0 temp 1 loss 7.43 pae 0.65 con 3.50 plddt 0.58 ptm 0.33 rg 28.66
4 models [0] recycles 0 hard 0 soft 0 temp 1 loss 7.33 pae 0.60 con 3.45 plddt 0.60 ptm 0.37 rg 28.72
5 models [3] recycles 0 hard 0 soft 0 temp 1 loss 6.76 pae 0.53 con 2.98 plddt 0.65 ptm 0.52 rg 28.93
6 models [0] recycles 0 hard 0 soft 0 temp 1 loss 6.55 pae 0.52 con 2.89 plddt 0.68 ptm 0.50 rg 28.16
7 models [2] recycles 0 hard 0 soft 0 temp 1 loss 6.34 pae 0.48 con 2.78 plddt 0.68 ptm 0.56 rg 27.74
8 models [2] recycles 0 hard 0 soft 0 temp 1 loss 6.13 pae 0.43 con 2.65 plddt 0.72 ptm 0.60 rg 27.66
9 models [0] recycles 0 hard 0 soft 0 temp 1 loss 5.72 pae 0.36 con 2.38 plddt 0.77 ptm 0.67 rg 27.50
10 models [1] recycles 0 hard 0 soft 0 temp 1 loss 5.86 pae 0.37 con 2.48 plddt 0

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()

['TAEEQMYEFCWRACRYGVGSRNGTPVDKDFWRKSPSTSLQDYRRWMLAILEYWCQRTNMAPYFDFDAFEEDTRNLSKEEWFEALADYITGALARKYDSALRQTLQACGYEEGKTYTRPGDEIRHCVRLWFYRASWWGFFRDYNLGSFAFRRLHRSAIAALYDPENNTFNVEGCRQAVQEHMPELAEAIYESLWWNFPQFQELPAEYNEIDPEVRPYRKKFAHCNYLRYQEWRSRAGYMPFNAHEWENCKK']

In [ ]:
#@markdown #Redesign with ProteinMPNN
num_seqs = 8 #@param ["8", "16", "32", "64"] {type:"raw"}
mpnn_sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
rm_aa = "C" #@param {type:"string"}
use_solubleMPNN = False #@param {type:"boolean"}
#@markdown - `mpnn_sampling_temp` - control diversity of sampled sequences. (higher = more diverse).
#@markdown - `rm_aa='C'` - do not use [C]ysteines.
#@markdown - `use_solubleMPNN` - use weights trained only on soluble proteins. See [preprint](https://www.biorxiv.org/content/10.1101/2023.05.09.540044v2).
#@markdown


In [ ]:
from colabdesign.shared.protein import alphabet_list as chain_list
mpnn_model = mk_mpnn_model()
mpnn_model.prep_inputs(pdb_filename=f"{af_model.protocol}.pdb",
                       chain=",".join(chain_list[:COPIES]),
                       homooligmer=COPIES>1,
                       rm_aa=rm_aa,
                       weights = "soluble" if use_solubleMPNN else"original")
out = mpnn_model.sample(num=num_seqs//8,
                        batch=8,
                        temperature=mpnn_sampling_temp)
for seq,score in zip(out["seq"],out["score"]):
  print(score,seq.split("/")[0])

0.8580595513835052 SPRAEVEAFLANATKNGVGSAELTPVDKAYFESNPEQNFETFKDYYIARLKEYAEEKNWLKFFDFDAFKEATKNLSFDEWFEAFAKFVADALSKAFDAALEEALKKHGLEAGKVHKLPKEKAEAAVAYFKRLADLSTFFENYNLGAEAYKAMKDAALKALYDPETNSINPAAAKKAWDDAKPALAKAIYESLKKSYEAFKTFPAEYAALPAEARPLLEELGAANRLKRLKELAAEGWVEFDEAALAEYEA
0.8438086035808163 SPLDQVKAFLARAAAEGVGSLELTPTDEAYFASNPEQNFETFREYILARLREYAERHGRLAFFDFDAFAAATKNLSKDEFFAALAKFIADAYTKAFDAALDAALAAAGLAPGEVATLPREEALAAVRRFDRLADLSEFFDNYALGAAAYAAMRAAALAALYDPATSSIDPAAARAAWAAAAPALAAAIEASLKKSYEAFKTFPAEYAALPAEARPLLKTLGEANRLKRLAELAARGEVEFDEEKLARYLA
0.8339584293031895 TPEEAVRAFLADAAAAGVASRTLTPVDAAWFESAPETNLATFRDYLLARLREYAAERNWLAHFDFDAFRAATADLSPDEWFAALAAFIAAALTAAFAAALDATLAAHGMTPGAVHTLPPAAAAAAVATWRDLADLSDFFDDLALGAAAYAAMAAAAEAALYDPATSSVDAAAARAAWAAAAPALAAAIEASLRASYEAFKTFPARWAALPPEARPLAAVLGERARLERLKELAAKGLVEFDEEKLKKYEE
0.8049213755106822 TPEAQVRAFLADAAEKGVGSRTLTPVDAAYFRANPEANFETFRKYILARLREYAEARGRLAFFDFDAFEKATKDLSKDEWFKALAKFIADALSKDFAAALDQTLAAHGLAPGEVATLPPEAAAAALADFRRLADLSEFFDDYNLGAAAYAAMAAAAEAALYDPAAAAIDPA

In [ ]:
print("# rmsd tmscore sequence")
best = {}
best_rmsd = None
for n,seq in enumerate(out["seq"]):
  x = seq.split("/")[0]
  with open(f"{af_model.protocol}.esmfold.{n}.pdb","w") as handle:
    pdb_str = esmfold_api(x)
    handle.write(pdb_str)
  o = tmscore(f"{af_model.protocol}.pdb",
              f"{af_model.protocol}.esmfold.{n}.pdb")
  print(n,o["rms"],o["tms"],x)
  if best_rmsd is None or o["rms"] < best_rmsd:
    best_rmsd = o["rms"]
    best = {**o,"seq":x}

# rmsd tmscore sequence


[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 183M/183M [00:00<00:00, 239Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205


NetworkError: Protocol error (Page.navigate): Invalid parameters Failed to deserialize params.url - BINDINGS: string value expected at position 12

In [ ]:
print(best)

In [ ]:
#{'rms': 2.881, 'tms': 0.892, 'gdt': 0.803, 'seq': 'MTVAPPRVLRVRINEKNYILFSVFFETDENGNHAFRVKVWLFANIELDPAHANDPAYRASLFAEAASHPEVVIKFLESLVEYVKRLRKENPALSGDTVFLFILSLVVDYYNPDAPPGGNVPLTPEDLEAAKLILKEARKFLEELKKLLKELAEKINKTPGFSAEIEELTPERFKDLAKARMPQTPVSPNRRIYMLLTVVVSDKETWNNKEYRNSLLNEAKSGKERPQPEDFGPTTEITLEEFLAGFKWQP'}
#With no PDB ^^^^^^^^^
#With PBD 1
#('best' function not working) -> highest test score: LSLKLLEKALSKELADKIITFHLLGLVLEVSKDHPEKPIFDELRERLEELEEELEEHLDLPEEEFNELVDKRLSEFIAEAFSHPAVVDAFLDLLVTLKAMADIRSARAEKADEERRANDPSGKEEEVPDSEELVTLKKASAALDRALDTLLKDPRVREMVERYLRARGVRIPEEALSLPRAEQLRLAFQRLVAREAARMTPAGKTAAEVTDEELAASFAASPNPFARAFAHRFPELAKELAEMQDLHDKL

#With PDB 2

#SPRAEVEAFLANATKNGVGSAELTPVDKAYFESNPEQNFETFKDYYIARLKEYAEEKNWLKFFDFDAFKEATKNLSFDEWFEAFAKFVADALSKAFDAALEEALKKHGLEAGKVHKLPKEKAEAAVAYFKRLADLSTFFENYNLGAEAYKAMKDAALKALYDPETNSINPAAAKKAWDDAKPALAKAIYESLKKSYEAFKTFPAEYAALPAEARPLLEELGAANRLKRLKELAAEGWVEFDEAALAEYEA
#SPLDQVKAFLARAAAEGVGSLELTPTDEAYFASNPEQNFETFREYILARLREYAERHGRLAFFDFDAFAAATKNLSKDEFFAALAKFIADAYTKAFDAALDAALAAAGLAPGEVATLPREEALAAVRRFDRLADLSEFFDNYALGAAAYAAMRAAALAALYDPATSSIDPAAARAAWAAAAPALAAAIEASLKKSYEAFKTFPAEYAALPAEARPLLKTLGEANRLKRLAELAARGEVEFDEEKLARYLA
#TPEEAVRAFLADAAAAGVASRTLTPVDAAWFESAPETNLATFRDYLLARLREYAAERNWLAHFDFDAFRAATADLSPDEWFAALAAFIAAALTAAFAAALDATLAAHGMTPGAVHTLPPAAAAAAVATWRDLADLSDFFDDLALGAAAYAAMAAAAEAALYDPATSSVDAAAARAAWAAAAPALAAAIEASLRASYEAFKTFPARWAALPPEARPLAAVLGERARLERLKELAAKGLVEFDEEKLKKYEE
#TPREEVEKFLKEAAANGVGSLELTPVDLAYFKSQPEANFATFREYLLARLREYAERHNRLPFFDFDAFAAATADLSLDAWFAAFAAFIAAAYEKAFDAALDATLAAAGLTAGEVATLPPAAAAAAVARFRQLADLTEFFENYALGAAAYKAMLDAALAALYDPATSSIDAAAARAAWAAAAPALAAAIEASLKKTYEAFLKFPAEYAKLPAEARPLLEELGERNRLKRLKELAEQGLVDFDEEKLAKYEA
#SPLAAVKAFLEEGAANGVASRTLTPVDLEYFASAPEQNLETFREYILARLREYAAERNWLAFFDFDAFAAATADLSPDEWFRALAKFIADALRAAFRAALAATLAAAGLAPGEVATLPAAAALAALATFRRLADLSEFFENRALGAAAYAAMAAAAEAALYDPAANAIDAAAALAAWDAAAPALEAAIYESLKRSYEAFLTAPARWAALPPEARPLLATLGEANRLKRLAALAAEGWVDFDEEALARYEA
#SPLDEVRAFLDRGAAEGVLSRELTPVDAEYLAAQPERNLATFRDYVLARLKEYAERHGRLAFFDFDAFRAATADLSLDAWFAALAKFIADALGAAFDAALDAALAAAGLAPGEVATLPPAEAAAAVAAFRRLASLAEFFDGRALGAEAAAAAAAAALAALYDAAAAAIDAAAARAAWAAAAPALAAAIEASLRRSYEAFLRFPAEYAALPPEARPLLATLGEANRLLRDAELAAAGEVAFDEAALAAALA
#TPEAEVRAFLERGAKNGVTSRRLLPTDAAYLASQPEQNLATFRDYLLARLREYAEEHNRLAFFDFDAFAKATADLSFDEWFAAFAKFVAESLGAAFDAALDAALKAHGLAAGQVHTLPAAQAQAAVAHFDALADLAEFFEDLNLGAAAYAAMRAAALAALYDPATSSIDPAAARAAWAAARPALAAAIEASLKKSYEAFLTFPAEWAALDPAARPLAETLGRANRLKRAAELAAQGRVEFDEAALAEAEA

#With scores:
'''
0.8580595513835052 SPRAEVEAFLANATKNGVGSAELTPVDKAYFESNPEQNFETFKDYYIARLKEYAEEKNWLKFFDFDAFKEATKNLSFDEWFEAFAKFVADALSKAFDAALEEALKKHGLEAGKVHKLPKEKAEAAVAYFKRLADLSTFFENYNLGAEAYKAMKDAALKALYDPETNSINPAAAKKAWDDAKPALAKAIYESLKKSYEAFKTFPAEYAALPAEARPLLEELGAANRLKRLKELAAEGWVEFDEAALAEYEA
0.8438086035808163 SPLDQVKAFLARAAAEGVGSLELTPTDEAYFASNPEQNFETFREYILARLREYAERHGRLAFFDFDAFAAATKNLSKDEFFAALAKFIADAYTKAFDAALDAALAAAGLAPGEVATLPREEALAAVRRFDRLADLSEFFDNYALGAAAYAAMRAAALAALYDPATSSIDPAAARAAWAAAAPALAAAIEASLKKSYEAFKTFPAEYAALPAEARPLLKTLGEANRLKRLAELAARGEVEFDEEKLARYLA
0.8339584293031895 TPEEAVRAFLADAAAAGVASRTLTPVDAAWFESAPETNLATFRDYLLARLREYAAERNWLAHFDFDAFRAATADLSPDEWFAALAAFIAAALTAAFAAALDATLAAHGMTPGAVHTLPPAAAAAAVATWRDLADLSDFFDDLALGAAAYAAMAAAAEAALYDPATSSVDAAAARAAWAAAAPALAAAIEASLRASYEAFKTFPARWAALPPEARPLAAVLGERARLERLKELAAKGLVEFDEEKLKKYEE
0.8049213755106822 TPEAQVRAFLADAAEKGVGSRTLTPVDAAYFRANPEANFETFRKYILARLREYAEARGRLAFFDFDAFEKATKDLSKDEWFKALAKFIADALSKDFAAALDQTLAAHGLAPGEVATLPPEAAAAALADFRRLADLSEFFDDYNLGAAAYAAMAAAAEAALYDPAAAAIDPAAARAAWAAAAPALAAAIEASLKKSYEAFLKLPAEYAALPPEARPLLERLGAANRLEREAALAAEGWVDFDEAALAAARA
0.8289269329275916 TPREEVEKFLKEAAANGVGSLELTPVDLAYFKSQPEANFATFREYLLARLREYAERHNRLPFFDFDAFAAATADLSLDAWFAAFAAFIAAAYEKAFDAALDATLAAAGLTAGEVATLPPAAAAAAVARFRQLADLTEFFENYALGAAAYKAMLDAALAALYDPATSSIDAAAARAAWAAAAPALAAAIEASLKKTYEAFLKFPAEYAKLPAEARPLLEELGERNRLKRLKELAEQGLVDFDEEKLAKYEA
0.8456560373564058 SPLAAVKAFLEEGAANGVASRTLTPVDLEYFASAPEQNLETFREYILARLREYAAERNWLAFFDFDAFAAATADLSPDEWFRALAKFIADALRAAFRAALAATLAAAGLAPGEVATLPAAAALAALATFRRLADLSEFFENRALGAAAYAAMAAAAEAALYDPAANAIDAAAALAAWDAAAPALEAAIYESLKRSYEAFLTAPARWAALPPEARPLLATLGEANRLKRLAALAAEGWVDFDEEALARYEA
0.7986974025049212 SPLDEVRAFLDRGAAEGVLSRELTPVDAEYLAAQPERNLATFRDYVLARLKEYAERHGRLAFFDFDAFRAATADLSLDAWFAALAKFIADALGAAFDAALDAALAAAGLAPGEVATLPPAEAAAAVAAFRRLASLAEFFDGRALGAEAAAAAAAAALAALYDAAAAAIDAAAARAAWAAAAPALAAAIEASLRRSYEAFLRFPAEYAALPPEARPLLATLGEANRLLRDAELAAAGEVAFDEAALAAALA
0.831556801375267 TPEAEVRAFLERGAKNGVTSRRLLPTDAAYLASQPEQNLATFRDYLLARLREYAEEHNRLAFFDFDAFAKATADLSFDEWFAAFAKFVAESLGAAFDAALDAALKAHGLAAGQVHTLPAAQAQAAVAHFDALADLAEFFEDLNLGAAAYAAMRAAALAALYDPATSSIDPAAARAAWAAARPALAAAIEASLKKSYEAFLTFPAEWAALDPAARPLAETLGRANRLKRAAELAAQGRVEFDEAALAEAEA
'''